In [258]:
import json
import pandas as pd
from pandas import json_normalize
from collections import defaultdict


In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

tournaments = pd.read_json('./tournaments.json')
teams = pd.read_json('./teams.json')
players = pd.read_json('./players.json')

## Number of total tournaments from 2020 to 2023  (248)

In [6]:
tournaments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         248 non-null    int64 
 1   leagueId   248 non-null    int64 
 2   name       248 non-null    object
 3   slug       248 non-null    object
 4   sport      248 non-null    object
 5   startDate  248 non-null    object
 6   endDate    248 non-null    object
 7   stages     248 non-null    object
dtypes: int64(2), object(6)
memory usage: 15.6+ KB


## Number of tournaments each years

In [7]:
tournaments_2023 = tournaments[tournaments['startDate'].str.startswith('2023')] # 75
tournaments_2022 = tournaments[tournaments['startDate'].str.startswith('2022')] # 82
tournaments_2021 = tournaments[tournaments['startDate'].str.startswith('2021')] # 64
tournaments_2020 = tournaments[tournaments['startDate'].str.startswith('2020')] # 27


## Extra need datas

In [267]:
tournaments = pd.read_json('./tournaments.json')
mapping = pd.read_json('./mapping_data.json')

stages = tournaments['stages']

sections = []
for array in stages:
    for obj in array:
        if 'sections' in obj:
            sections.append(obj['sections'])
                        
matches = []
for array in sections:
    for obj in array:
        if 'matches' in obj:
            matches.append(obj['matches'])
            
teams = []
for array in matches:
    for obj in array:
        if 'teams' in obj:
            teams.append(obj['teams'])

            
players = []
for array in teams:
    for obj in array:
        if 'players' in obj:
            players.append(obj['players'])

games = []
for array in matches:
    for obj in array:
        if 'games' in obj:
            for game in obj['games']:
                if 'id' in game:
                    game['game_id'] = game.pop('id')
                if game.get('state') == 'completed':
                    # Remove the 'number' and 'state' fields
                    game.pop('number', None)
                    game.pop('state', None)
                    
                    # Update 'result' to a string
                    if 'teams' in game:
                        for team in game['teams']:
                            team['result'] = team['result']['outcome']
                    games.append(game)
                    
# extra game                    
# output_file = 'games.json'

# # Write the list to a JSON file
# with open(output_file, 'w') as json_file:
#     json.dump(games, json_file, indent=None)
    
# len(stages) #602
# len(sections) #507      
# len(teams) #19386
# len(players) #38772
# len(games) #19386

# mapping.shape (25283, 4)
participantList = [item for item in mapping['participantMapping']]

# top 1 & 6
# mid 8 & 3
# juggle 2 & 7
# bot 9 & 4
# support 5 & 10

key_to_role = {
    '1': 'top',
    '2': 'jungle',
    '3': 'mid',
    '4': 'bot',
    '5': 'support',
    '6': 'top',
    '7': 'jungle',
    '8': 'mid',
    '9': 'bot',
    '10': 'support'
}

# Create a dictionary to store the counts
key_value_counts = defaultdict(int)

# Iterate through the list of dictionaries
for data_dict in participantList:
    for key, value in data_dict.items():
        key_value_counts[(key, value)] += 1

roles_array = []        
for (key, value), count in key_value_counts.items():
    role = key_to_role.get(key, key) # Use the role name if found, or the key itself
    result_dict = {"role": role, "player_id": value, "count": count}
    roles_array.append(result_dict)
    
    
# # extra roles  
# output_file_path = 'role_datas.json'

# # Write the data to the JSON file
# with open(output_file_path, 'w') as json_file:
#     json.dump(roles_array, json_file)


[{'role': 'mid', 'player_id': '107492116585043595', 'count': 83},
 {'role': 'support', 'player_id': '105388980252039870', 'count': 56},
 {'role': 'support', 'player_id': '110730240651882119', 'count': 14},
 {'role': 'jungle', 'player_id': '108366332471078988', 'count': 69},
 {'role': 'top', 'player_id': '108205130568869560', 'count': 40},
 {'role': 'bot', 'player_id': '106267600924403194', 'count': 111},
 {'role': 'jungle', 'player_id': '106267599829820917', 'count': 115},
 {'role': 'mid', 'player_id': '107492063150003806', 'count': 106},
 {'role': 'top', 'player_id': '105501816646382923', 'count': 112},
 {'role': 'bot', 'player_id': '102483272156027229', 'count': 163},
 {'role': 'mid', 'player_id': '107560281476330464', 'count': 54},
 {'role': 'support', 'player_id': '107559633217576287', 'count': 19},
 {'role': 'support', 'player_id': '105397238669368241', 'count': 37},
 {'role': 'jungle', 'player_id': '103478281341350420', 'count': 86},
 {'role': 'top', 'player_id': '103478281338008